# Drive mounting and imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install codecarbon
!pip install keras-flops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 174 kB 35.5 MB/s 
     |████████████████████████████████| 99 kB 5.0 MB/s 
     |████████████████████████████████| 46 kB 5.2 MB/s 
     |████████████████████████████████| 64 kB 3.2 MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=9b495d61aac3710ee10a0a929cd8bf31bdc454c8e281b98409bc47a0582c85f9
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
Successfully built py-cpuinfo
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 462 kB 41.4 MB/s 


In [5]:
import os

os.chdir('/content/drive/MyDrive/Q8/I2RCED/project_MYM')
os.getcwd()

'/content/drive/MyDrive/Q8/I2RCED/project_MYM'

In [ ]:
!python chesslive-codecarbon.py

170508288/170498071 [==============================] - 13s 0us/step
2022-05-30 15:00:25.138900: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
58900480/58889256 [==============================] - 0s 0us/step
[codecarbon INFO @ 15:00:26] [setup] RAM Tracking...
[codecarbon INFO @ 15:00:26] [setup] GPU Tracking...
[codecarbon INFO @ 15:00:26] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 15:00:26] [setup] CPU Tracking...
[codecarbon WARNING @ 15:00:26] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 15:00:27] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 15:00:27] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 15:00:27] >>> Tracker's metadata:
[codecarbon INFO @ 15:00:27]   Platform system: Linux-5

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.datasets import cifar10, mnist
from keras.preprocessing.image import img_to_array, array_to_img
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.xception import Xception
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.imagenet_utils import decode_predictions
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D
from keras.models import Model
from codecarbon import EmissionsTracker
from keras_flops import get_flops

# Input Data and Data Augmentation

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
# Load the CIFAR10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input image dimensions and classes
input_shape = x_train.shape[1:]
num_classes = 10

# Target one-hot vectorization
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Normalize data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

print(input_shape)

(32, 32, 3)


In [ ]:
# Load the MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.dstack([x_train] * 3)
x_train = x_train.reshape(-1, 28,28,3)
x_train = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in x_train])
x_test = np.dstack([x_test] * 3)
x_test = x_test.reshape(-1, 28,28,3)
x_test = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in x_test])

# Input image dimensions and classes
input_shape = x_train.shape[1:]
num_classes = 10

# Target one-hot vectorization
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Normalize data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

print(input_shape)

11501568/11490434 [==============================] - 0s 0us/step
(48, 48, 3)


In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False
)  # randomly flip images
        
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Models

In [ ]:
# Parameters 
batch_size = 32
epochs = 50
optimizer = 'adam'

## VGG16

In [ ]:
# Import pre-trained layers
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_shape) 
 
# Freeze convolutional layers
for layer in base_model_vgg16.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model_vgg16.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(4096, activation='relu')(x) # number of layers and units are hyperparameters, as usual
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x) # should match # of classes predicted

# This is the model we will train
model_vgg16 = Model(inputs=base_model_vgg16.input, outputs=predictions)
model_vgg16.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_vgg16.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0   

## VGG19


In [ ]:
# Import pre-trained layers
base_model_vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=input_shape) 
 
# Freeze convolutional layers
for layer in base_model_vgg19.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model_vgg19.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(4096, activation='relu')(x) # number of layers and units are hyperparameters, as usual
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model_vgg19 = Model(inputs=base_model_vgg19.input, outputs=predictions)
model_vgg19.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_vgg19.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0   

## ResNet

In [ ]:
# Import pre-trained layers
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape) 
 
# Freeze convolutional layers
for layer in base_model_resnet50.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model_resnet50.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(4096, activation='relu')(x) # number of layers and units are hyperparameters, as usual
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x) # should match # of classes predicted

# This is the model we will train
model_resnet50 = Model(inputs=base_model_resnet50.input, outputs=predictions)
model_resnet50.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_resnet50.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_7[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                            

# [kera-flops](https://pypi.org/project/keras-flops/)

In [ ]:
# Models
flops = get_flops(model_vgg16, batch_size=batch_size)
print(f"FLOPS VGG16:    {flops / 10 ** 9:.03} G - {flops}")

flops = get_flops(model_vgg19, batch_size=batch_size)
print(f"FLOPS VGG19:    {flops / 10 ** 9:.03} G - {flops}")

flops = get_flops(model_resnet50, batch_size=batch_size)
print(f"FLOPS ResNet50: {flops / 10 ** 9:.03} G - {flops}")

FLOPS VGG16:    46.3 G - 46339966848
FLOPS VGG19:    58.6 G - 58572474240
FLOPS ResNet50: 16.3 G - 16289166848


In [ ]:
# Trainable Layer
flops = get_flops(model_vgg16, batch_size=batch_size) - get_flops(base_model_vgg16, batch_size=batch_size)
print(f"FLOPS VGG16:    {flops / 10 ** 9:.03} G - {flops}")

flops = get_flops(model_vgg19, batch_size=batch_size) - get_flops(base_model_vgg19, batch_size=batch_size)
print(f"FLOPS VGG19:    {flops / 10 ** 9:.03} G - {flops}")

flops = get_flops(model_resnet50, batch_size=batch_size) - get_flops(base_model_resnet50, batch_size=batch_size)
print(f"FLOPS ResNet50: {flops / 10 ** 9:.03} G - {flops}")

FLOPS VGG16:    1.21 G - 1210845056
FLOPS VGG19:    1.21 G - 1210845056
FLOPS ResNet50: 3.22 G - 3224110976


# [Codecarbon](https://github.com/mlco2/codecarbon)

In [ ]:
tracker = EmissionsTracker()

# Train VGG16
for i in range(3):
  tracker.start()
  history_vgg16 = model_vgg16.fit_generator(
      datagen.flow(x_train, y_train, batch_size=batch_size), 
      epochs=epochs,
      verbose = 1,
      validation_data = (x_test, y_test)
  )
  tracker.stop()

[codecarbon INFO @ 17:00:58] [setup] RAM Tracking...
[codecarbon INFO @ 17:00:58] [setup] GPU Tracking...
[codecarbon INFO @ 17:00:58] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:00:58] [setup] CPU Tracking...
[codecarbon WARNING @ 17:00:58] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 17:00:59] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 17:00:59] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 17:00:59] >>> Tracker's metadata:
[codecarbon INFO @ 17:00:59]   Platform system: Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
[codecarbon INFO @ 17:00:59]   Python version: 3.7.13
[codecarbon INFO @ 17:00:59]   Available RAM : 12.683 GB
[codecarbon INFO @ 17:00:59]   CPU count: 2
[codecarbon INFO @ 17:00:59]   CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 17:00:59]   GPU count: 1
[codecarbon INFO @ 17:00:59]   GPU mode

Epoch 1/50
 124/1875 [>.............................] - ETA: 50s - loss: 1.0278 - accuracy: 0.6583

[codecarbon INFO @ 17:01:14] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.7559428215026855 W
[codecarbon INFO @ 17:01:14] Energy consumed for all GPUs : 0.000291 kWh. All GPUs Power : 69.891 W


 126/1875 [=>............................] - ETA: 50s - loss: 1.0234 - accuracy: 0.6592

[codecarbon INFO @ 17:01:14] Energy consumed for all CPUs : 0.000177 kWh. All CPUs Power : 42.5 W
[codecarbon INFO @ 17:01:14] 0.000488 kWh of electricity used since the begining.


...

1628/1875 [=========================>....] - ETA: 7s - loss: 0.4750 - accuracy: 0.8419

[codecarbon INFO @ 17:01:59] Energy consumed for RAM : 0.000079 kWh. RAM Power : 4.7559428215026855 W


1630/1875 [=========================>....] - ETA: 7s - loss: 0.4747 - accuracy: 0.8420

[codecarbon INFO @ 17:01:59] Energy consumed for all GPUs : 0.001135 kWh. All GPUs Power : 77.006 W
[codecarbon INFO @ 17:01:59] Energy consumed for all CPUs : 0.000708 kWh. All CPUs Power : 42.5 W
[codecarbon INFO @ 17:01:59] 0.001923 kWh of electricity used since the begining.


1682/1875 [=========================>....] - ETA: 5s - loss: 0.4713 - accuracy: 0.8430

In [ ]:
# Train VGG19
for i in range(3):
  tracker.start()
  history_vgg19 = model_vgg19.fit_generator(
      datagen.flow(x_train, y_train, batch_size=batch_size), 
      epochs=epochs,
      verbose = 1,
      validation_data = (x_test, y_test)
  )
  tracker.stop()

In [ ]:
# Train ResNet50
for i in range(3):
  tracker.start()
  history_resnet50 = model_resnet50.fit_generator(
      datagen.flow(x_train, y_train, batch_size=batch_size), 
      epochs=epochs,
      verbose = 1,
      validation_data = (x_test, y_test)
  )
  tracker.stop()